In [1]:
# collatz_happy_training.py
# ---------------------------------------
# Generate datasets + Train models for
# Collatz Conjecture & Happy Numbers
# ---------------------------------------

import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, mean_squared_error

# -----------------------------
# 1. Collatz sequence functions
# -----------------------------
def collatz_steps(n):
    steps = 0
    while n != 1 and steps < 1000:  # loop safeguard
        if n % 2 == 0:
            n = n // 2
        else:
            n = 3 * n + 1
        steps += 1
    return steps

# Generate Collatz dataset
collatz_nums = np.arange(2, 2000)
collatz_steps_list = [collatz_steps(x) for x in collatz_nums]

collatz_df = pd.DataFrame({
    "number": collatz_nums,
    "steps": collatz_steps_list,
    "long_seq": [1 if s > 100 else 0 for s in collatz_steps_list]  # classification label
})

collatz_df.to_csv("collatz_dataset.csv", index=False)

# Train models
X_col = collatz_df[["number"]]
y_steps = collatz_df["steps"]
y_class = collatz_df["long_seq"]

X_train, X_test, y_train, y_test = train_test_split(X_col, y_steps, test_size=0.2, random_state=42)
rfr = RandomForestRegressor(n_estimators=200, random_state=42)
rfr.fit(X_train, y_train)
print("Collatz RFR MSE:", mean_squared_error(y_test, rfr.predict(X_test)))
joblib.dump(rfr, "rfr_collatz.joblib")

X_train, X_test, y_train, y_test = train_test_split(X_col, y_class, test_size=0.2, random_state=42)
rfc = RandomForestClassifier(n_estimators=200, random_state=42)
rfc.fit(X_train, y_train)
print("Collatz RFC Accuracy:", accuracy_score(y_test, rfc.predict(X_test)))
joblib.dump(rfc, "rfc_collatz.joblib")

# -----------------------------
# 2. Happy numbers functions
# -----------------------------
def is_happy(n):
    seen = set()
    while n != 1 and n not in seen:
        seen.add(n)
        n = sum(int(d)**2 for d in str(n))
    return n == 1

happy_nums = np.arange(2, 2000)
happy_labels = [1 if is_happy(x) else 0 for x in happy_nums]
happy_df = pd.DataFrame({
    "number": happy_nums,
    "is_happy": happy_labels
})

happy_df.to_csv("happy_dataset.csv", index=False)

# Train models
X_happy = happy_df[["number"]]
y_happy = happy_df["is_happy"]

X_train, X_test, y_train, y_test = train_test_split(X_happy, y_happy, test_size=0.2, random_state=42)

log_reg = LogisticRegression(max_iter=500)
log_reg.fit(X_train, y_train)
print("Happy Logistic Accuracy:", accuracy_score(y_test, log_reg.predict(X_test)))
joblib.dump(log_reg, "log_happy.joblib")

rfc_happy = RandomForestClassifier(n_estimators=200, random_state=42)
rfc_happy.fit(X_train, y_train)
print("Happy RFC Accuracy:", accuracy_score(y_test, rfc_happy.predict(X_test)))
joblib.dump(rfc_happy, "rfc_happy.joblib")

print("✅ Training complete! Datasets + models saved locally.")


Collatz RFR MSE: 2100.4271471875
Collatz RFC Accuracy: 0.725
Happy Logistic Accuracy: 0.855
Happy RFC Accuracy: 0.745
✅ Training complete! Datasets + models saved locally.


In [5]:
from google.colab import files
files.download("collatz_dataset.csv")
files.download("rfr_collatz.joblib")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
import joblib

# Load trained models
rfr_collatz = joblib.load("rfr_collatz.joblib")
rfc_collatz = joblib.load("rfc_collatz.joblib")
log_happy = joblib.load("log_happy.joblib")
rfc_happy = joblib.load("rfc_happy.joblib")

# Function to generate Collatz sequence
def collatz_sequence(n):
    seq = [n]
    while n != 1:
        if n % 2 == 0:
            n = n // 2
        else:
            n = 3*n + 1
        seq.append(n)
    return seq

# Provide your number here
num = int(input("Enter a number: "))

# Collatz predictions
pred_steps = rfr_collatz.predict([[num]])[0]
pred_long_class = rfc_collatz.predict([[num]])[0]

# Happy number predictions
pred_happy_log = log_happy.predict([[num]])[0]
pred_happy_rf = rfc_happy.predict([[num]])[0]

# Generate full Collatz sequence
sequence = collatz_sequence(num)

print(f"\nCollatz sequence for {num}: {sequence}")
print(f"Predicted steps to reach 1: {pred_steps:.0f}")
print(f"Predicted long sequence? {'Yes' if pred_long_class else 'No'}")
print(f"Happy number prediction (Logistic): {'Yes' if pred_happy_log else 'No'}")
print(f"Happy number prediction (Random Forest): {'Yes' if pred_happy_rf else 'No'}")

Enter a number: 2

Collatz sequence for 2: [2, 1]
Predicted steps to reach 1: 3
Predicted long sequence? No
Happy number prediction (Logistic): No
Happy number prediction (Random Forest): No


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
